In [60]:
import numpy as np
import sympy as sm

In [1]:
def jordan_wigner_majorana(N):
    """
    Jordan-Wigner transformation for Majorana fermions.

    Parameters:
    - N: int, number of fermions

    Returns:
    - G_matrix: 2N x 2N matrix representing the Jordan-Wigner transformation
    """

    # Initialize the Pauli matrices
    sigma_x = np.array([[0, 1], [1, 0]])
    sigma_y = np.array([[0, -1j], [1j, 0]])
    sigma_z = np.array([[1, 0], [0, -1]])

    # Identity matrix
    I = np.eye(2)

    # Initialize the Jordan-Wigner matrix
    G_matrix = np.zeros((2*N, 2*N), dtype=complex)

    for i in range(N):
        # Construct the ith Majorana operator
        operator_i = np.kron(np.eye(2**i), np.kron(sigma_z, I))
        
        # Construct the product term for the Jordan-Wigner matrix
        product_term = np.eye(1)
        for j in range(N):
            if j < i:
                product_term = np.kron(product_term, sigma_z)
            elif j > i:
                product_term = np.kron(product_term, sigma_x - 1j * sigma_y)

        # Update the Jordan-Wigner matrix
        G_matrix += operator_i @ product_term

    return G_matrix


In [35]:
import numpy as np
from qutip import qeye, sigmax, sigmay, sigmaz, tensor, create, destroy

def Is(i, levels=2): return [qeye(levels) for j in range(0, i)]

def Sx(N, i): return tensor(Is(i) + [sigmax()] + Is(N - i - 1))
def Sy(N, i): return tensor(Is(i) + [sigmay()] + Is(N - i - 1))
def Sz(N, i): return tensor(Is(i) + [sigmaz()] + Is(N - i - 1))

def I(N): return Sz(N, 0)*Sz(N, 0)

def b_(N, Np, i): return tensor(Is(i, levels=Np+1) + [destroy(Np+1)] + Is(N - i - 1, levels=Np+1))
def bd(N, Np, i): return tensor(Is(i, levels=Np+1) + [create(Np+1)] + Is(N - i - 1, levels=Np+1))

def osum(lst): return np.sum(np.array(lst, dtype=object))

def oprd(lst, d=None):
    if len(lst) == 0:
        return d
    p = lst[0]
    for U in lst[1:]:
        p = p*U
    return p

# N is the exponent, L is the length of the chain
def opow(L, op, N): return oprd([op for i in range(N)])


def commutator(A, B):
    return A*B - B*A


def anticommutator(A, B):
    return A*B + B*A


def a_(N, n, Opers=None):
    Sa, Sb, Sc = Sz, Sx, Sy
    if Opers is not None:
        Sa, Sb, Sc = Opers
    return oprd([Sa(N, j) for j in range(n)], d=I(N))*(Sb(N, n) + 1j*Sc(N, n))/2.


def ad(N, n, Opers=None):
    Sa, Sb, Sc = Sz, Sx, Sy
    if Opers is not None:
        Sa, Sb, Sc = Opers
    return oprd([Sa(N, j) for j in range(n)], d=I(N))*(Sb(N, n) - 1j*Sc(N, n))/2.

In [50]:
commutator(a_(2, 0), a_(2, 0))

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

In [51]:
def testFermions(N, jw):
    (Sa, _), (Sb, _), (Sc, _) = jw
    Opers = Sa, Sb, Sc
    zero = 0.*I(N)
    # test all the pairs
    for n in range(N):
        a_n = a_(N, n, Opers=Opers)
        adn = ad(N, n, Opers=Opers)
        for np in range(N):
            a_np = a_(N, np, Opers=Opers)
            adnp = ad(N, np, Opers=Opers)
            assert anticommutator(a_n, a_np) == zero
            if n == np:
                assert anticommutator(a_n, adnp) == I(N)
            else:
                assert anticommutator(a_n, adnp) == zero
        assert a_n*a_n == zero
        assert adn*adn == zero

## measurement-only braiding, symbolisim

In [106]:
N = 6
S_num = 8
S = [sm.symbols(f"S_{i}") for i in range(1,S_num+1)]
gamma = [sm.symbols(f'\gamma_{i}') for i in range(1, N+1)]
M = sm.symbols('M')

def prj_meas(i, j, s):
    return (1 + 1j * S[s-1] * gamma[i-1] * gamma[j-1])

In [107]:
sm.Eq(M, prj_meas(2, 3, 1) * prj_meas(4, 5, 2)
     * prj_meas(1, 2, 3) * prj_meas(3, 4, 4)
     * prj_meas(2, 4, 5) * prj_meas(3, 6, 6)
     * prj_meas(2, 3, 7) * prj_meas(2, 5, 8))

Eq(M, (1.0*I*S_1*\gamma_2*\gamma_3 + 1)*(1.0*I*S_2*\gamma_4*\gamma_5 + 1)*(1.0*I*S_3*\gamma_1*\gamma_2 + 1)*(1.0*I*S_4*\gamma_3*\gamma_4 + 1)*(1.0*I*S_5*\gamma_2*\gamma_4 + 1)*(1.0*I*S_6*\gamma_3*\gamma_6 + 1)*(1.0*I*S_7*\gamma_2*\gamma_3 + 1)*(1.0*I*S_8*\gamma_2*\gamma_5 + 1))